In [1]:
import pyvespa_functions as pf
from build_dataset import make_food_dataset
import pandas as pd
from vespa.deployment import VespaDocker

package = pf.create_package(app_type="semantic-search")

vespa_docker = VespaDocker()
app = vespa_docker.deploy(application_package=package)

Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 10/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 15/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 20/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 25/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Application is up!
Finished deployment.


In [3]:
# tenta carregar o dataset do arquivo, se não conseguir, cria um novo
try:
    df = pd.read_csv('input/food_dataset.csv')
except FileNotFoundError:
    types = {
        "contributor_id": "string",
        "name": "string",
        "id": "string",
        "minutes": "int",
        "tags": "string",
        "nutrition": "string",
        "n_steps": "int",
        "n_ingredients": "int",
        "steps": "string",
        "description": "string",
        "ingredients": "string",
        "submitted": "string"
    }

    df = pd.read_csv('archive/RAW_recipes.csv', dtype=types)
    df = df.dropna()
    df = df.reset_index(drop=True)
    df_reviews = pd.read_csv('archive/RAW_interactions.csv')
    df_reviews = df_reviews.dropna()
    df_reviews = df_reviews.reset_index(drop=True)

    # df = df[:5000]
    df = make_food_dataset(df, df_reviews) # add the body for each recipe
    df = df.dropna()
    df = df.reset_index(drop=True)
    # selecting only the body and id columns
    df.rename(columns={'recipe_body': 'body'}, inplace=True)
    df = df[['id', 'body', 'title']]
    # save the df as a csv file
    df.to_csv('input/food_dataset.csv', index=False)

df = df[197319:-1]
df.head()

,id,body,title,text
197319,354704,stromboli ham and veggie\n\nRecipe posted on:...,stromboli ham and veggie,stromboli ham and veggie stromboli ham and v...
197320,127437,stromgren swedish meatballs\n\nRecipe posted o...,stromgren swedish meatballs,stromgren swedish meatballs stromgren swedish ...
197321,273992,strong hot chocolate\n\nRecipe posted on: 2007...,strong hot chocolate,strong hot chocolate strong hot chocolate\n\nR...
197322,143903,stroopwafels\n\nRecipe posted on: 2005-11-05\n...,stroopwafels,stroopwafels stroopwafels\n\nRecipe posted on:...
197323,259153,strozzapreti or priest chokers with sage sau...,strozzapreti or priest chokers with sage sauce,strozzapreti or priest chokers with sage sau...


In [2]:
# number of fed documents
documents = app.query(yql='select * from sources * where true')
if documents.number_documents_indexed > 0:
    print(f"Number of documents fed: {documents.number_documents_indexed}")
else:
    feeder = pf.VespaFeeder(app)
    feeder.feed(df)
    
    documents = app.query(yql='select * from sources * where true')
    print(f"Number of documents fed: {documents.number_documents_indexed}")


Number of documents fed: 226654


In [8]:
from vespa.io import VespaQueryResponse

with app.syncio(connections=1) as session:
  query = "something spicy"
  response:VespaQueryResponse = session.query(
    yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding,q)) limit 5", 
    query=query, 
    ranking="semantic", 
    body = {
      "input.query(q)": f"embed({query})"
    }
  )
  assert(response.is_successful())

  for hit in response.hits:
    record = {}
    for field in ['id', 'title', 'body']:
        record[field] = hit['fields'][field]
    print(record)

{'id': '25625', 'title': 'now don t go  eeewww    before you try this', 'body': 'now don t go  eeewww    before you try this\n\nRecipe posted on: 2002-04-18\n\nTags: 15-minutes-or-less, time-to-make, course, main-ingredient, cuisine, preparation, occasion, north-american, for-1-or-2, low-protein, healthy, 5-ingredients-or-less, breakfast, lunch, main-dish, american, easy, beginner-cook, diabetic, fall, finger-food, kid-friendly, low-fat, summer, vegan, vegetarian, dietary, one-dish-meal, low-sodium, low-cholesterol, seasonal, sandwiches, low-saturated-fat, low-calorie, comfort-food, low-carb, inexpensive, northeastern-united-states, low-in-something, taste-mood, number-of-servings, presentation\n\nDescription: the only way to eat fresh tomatoes! in anticipation of the impending tomato harvest, i share with you a family secret. this is the reason we look forward to the end of summer, to the time when our gardens and our friends gardens are bursting with little red-skinned treasures. whe

In [6]:
from vespa.io import VespaQueryResponse

with app.syncio(connections=1) as session:
    response:VespaQueryResponse = session.query(
        yql="select * from sources * where userQuery() limit 5", 
        query="something with no meat", 
        ranking="bm25"
    )
assert(response.is_successful())

for hit in response.hits:
    record = {}
    for field in ['id', 'title', 'body']:
        record[field] = hit['fields'][field]
    print(record)

{'id': '67685', 'title': 'grilled meat skewers with bay leaves', 'body': 'grilled <hi>meat</hi> skewers <hi>with</hi> bay leaves\n\nRecipe posted on: 2003-07-28\n\nTags: 30-minutes-or-less, time-to-make, course, main-ingredient, preparation, very-low-carbs, main-dish, beef, pork, barbecue, dietary, low-sodium, high-protein, low-carb, high-in-<hi>something</hi>, low-in-<hi>something</hi>, <hi>meat</hi>, equipment, grilling\n\nDescription: this recipe is from the houston chronicle. the smell of the bay leaves, while grilling, is wonderful. throw some on the fire, too, for great aroma-therapy! the time indicated does not include time to prepare the grill.\n\nThis recipe takes 30 minutes to be done.\n\nFor this recipe you will need the ingredients: \npork\nextra virgin olive oil\nsalt & freshly ground black pepper\ngarlic\nbay leaves\n\nThe 7 steps to make this recipe are: \nprepare the grill for a moderately hot fire and place rack about 4 inches from heat\nif using wood skewers \nsoak in

In [4]:
# loading the Questions.xlsx and answering each question query
questions = pd.read_excel('input/Questions.xlsx')
questions

,Tipo,Descrição,Query
0,Keywords,Pergunta simples,chocolate cake recipe
1,Keywords,Pergunta simples,strogonoff with rice
2,Keywords,Pergunta simples,fresh lemonade
3,Semantica,Pergunta média,pasta without eggs
4,Keywords,Pergunta simples,apple pie
5,Keywords,Pergunta simples,Brûlée Cream
6,Semantica,Pergunta média,how to make a pizza without an oven
7,Semantica,Pergunta média,pancake without flour and milk
8,Semantica,Pergunta difícil,healthy recipe for quick lunch
9,Semantica,Pergunta difícil,what can I make for a romantic dinner


In [7]:

data = pd.DataFrame(columns=['id', 'title', 'body', 'Query', 'Tipo', 'Descrição'])

i=0
for input_query in questions['Query']:

    # save a checkpoint each 100 queries
    if i % 100 == 0:
        data.to_excel('output/Results_hybrid.xlsx', index=False)

    with app.syncio(connections=1) as session:
        response:VespaQueryResponse = session.query(
            yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding,q)) limit 5", 
            query=input_query, 
            ranking="fusion", 
            body = {
            "input.query(q)": f"embed({input_query})"
            }
        )
    assert(response.is_successful())
    
    for hit in response.hits:
        record = {}
        for field in ['id', 'title', 'body']:
            record[field] = hit['fields'][field]
        record["Query"] = input_query
        record["Tipo"] = questions[questions['Query'] == input_query]['Tipo'].values[0]
        record["Descrição"] = questions[questions['Query'] == input_query]['Descrição'].values[0]
        data = pd.concat([data, pd.DataFrame([record])])

    i+=1

# Sorting
data = data.sort_values(by=['Tipo', 'Query'])

# reordering columns
data = data[['Tipo', 'Descrição', 'Query', 'id', 'title', 'body']]

# exporting to excel
data.to_excel('output/Results_hybrid.xlsx', index=False)

In [11]:

# data = pd.DataFrame(columns=['id', 'title', 'body', 'Query', 'Tipo', 'Descrição'])

# i=0
# for input_query in questions['Query']:

#     # save a checkpoint each 100 queries
#     if i % 100 == 0:
#         data.to_excel('output/Results_semantic.xlsx', index=False)

#     with app.syncio(connections=1) as session:
#         response:VespaQueryResponse = session.query(
#             yql="select * from sources * where ({targetHits:1000}nearestNeighbor(embedding,q)) limit 5", 
#             query=input_query, 
#             ranking="semantic", 
#             body = {
#             "input.query(q)": f"embed({input_query})"
#             }
#         )
#     assert(response.is_successful())
    
#     for hit in response.hits:
#         record = {}
#         for field in ['id', 'title', 'body']:
#             record[field] = hit['fields'][field]
#         record["Query"] = input_query
#         record["Tipo"] = questions[questions['Query'] == input_query]['Tipo'].values[0]
#         record["Descrição"] = questions[questions['Query'] == input_query]['Descrição'].values[0]
#         data = pd.concat([data, pd.DataFrame([record])])

#     i+=1

# # Sorting
# data = data.sort_values(by=['Tipo', 'Query'])

# # reordering columns
# data = data[['Tipo', 'Descrição', 'Query', 'id', 'title', 'body']]

# # exporting to excel
# data.to_excel('output/Results_semantic.xlsx', index=False)

In [12]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import linear_kernel

# def setup_tfidf(documents):
#     # Combine title and body for TF-IDF processing
#     documents['text'] = documents['title'] + " " + documents['body']
    
#     # Create the TF-IDF vectorizer and fit to the document texts
#     tfidf_vectorizer = TfidfVectorizer(stop_words='english')
#     tfidf_matrix = tfidf_vectorizer.fit_transform(documents['text'])
    
#     return tfidf_vectorizer, tfidf_matrix

# def find_top_hits(tfidf_vectorizer, tfidf_matrix, query, top_n=5):
#     # Transform the query to the same TF-IDF vector space as the documents
#     query_tfidf = tfidf_vectorizer.transform([query])
    
#     # Calculate cosine similarities between the query and all documents
#     cosine_similarities = linear_kernel(query_tfidf, tfidf_matrix).flatten()
    
#     # Get the top N matching documents
#     top_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
#     return top_indices

# documents = df

# # Set up TF-IDF
# tfidf_vectorizer, tfidf_matrix = setup_tfidf(documents)

# # DataFrame to store results
# data_tfidf = pd.DataFrame()

# for input_query in questions['Query']:
#     # Find top hits for the query
#     top_indices = find_top_hits(tfidf_vectorizer, tfidf_matrix, input_query)
    
#     for index in top_indices:
#         record = {}
#         # Extract details from documents
#         for field in ['id', 'title', 'body']:
#             record[field] = documents.loc[index, field]
#         # Add query details
#         record["Query"] = input_query
#         record["Tipo"] = questions[questions['Query'] == input_query]['Tipo'].values[0]
#         record["Descrição"] = questions[questions['Query'] == input_query]['Descrição'].values[0]
        
#         # Append to the result DataFrame
#         data_tfidf = pd.concat([data_tfidf, pd.DataFrame([record])])


# # Sorting
# data_tfidf = data_tfidf.sort_values(by=['Tipo', 'Query'])

# # reordering columns
# data_tfidf = data_tfidf[['Tipo', 'Descrição', 'Query', 'id', 'title', 'body']]

# # exporting to excel
# # data_tfidf.to_excel('output/Results_Tfidf.xlsx', index=False)


In [13]:
# make a file named "output/done.txt" containing the word "done" to indicate that the script has finished

with open("output/done.txt", "w") as f:
    f.write("done")